# M2 Rigid Body Position

This Jupyter notebook performs Rigid Body Position (RBP) displacements of the M2 and verify how the force on the axial and tangent actuators change.
The M2 Rigid Body Movement (RBM) along each Degree of Freedom (DoF) is actuated individually with the target position query to the user.

In addition, a 'Start' input is query to the user (response either y [yes] or n [no]) with a timeout exit in order to avoid endless execution of the script.

The `move_m2_rbp()` defined below move the M2 as rigid body using the `m2.cmd_positionMirror.set_start()` function and controlling new telemetry using the `seqNum` attribute, an integer number in the telemetry call attributes that increment only when new telemetry data are retrived.

The first check is on the M2 settle condition after the movement, checking the steps value of the hardpoints, retrieved by the function `get_hardpoint_steps()` defined below.

After the displacement another check is perfomed on the forces of all actuators, using the following telemetry calls:

    - m2.tel_forceErrorTangent.get().force
    - m2.tel_tangentForce.get().measured
    - m2.tel_axialForce.get().measured
    - m2.tel_forceErrorTangent.get().sum
    - m2.tel_forceErrorTangent.get().weight

The log of this script is managed by the `MyLogger` Class that send the output both at the screen and into a [isoTdatetime]_logfile.txt

# Important: 
In this script the function `m2.cmd_positionMirror.set_start()` is used to move the M2. Any D.O.F that is not declared will be set to the home position (e.g. using `m2.cmd_positionMirror.set_start()` the M2 is moved to the home position, `m2.cmd_positionMirror.set_start(x=position)` will move the M2 along the x-axis of `position` and the other axis will set at the home position).  

## Import Modules

In [ ]:
import asyncio
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com.constant import (
    NUM_ACTUATOR,
    NUM_TANGENT_LINK,
    TANGENT_LINK_LOAD_BEARING_LINK,
    TANGENT_LINK_THETA_Z_MOMENT,
    TANGENT_LINK_TOTAL_WEIGHT_ERROR,
    LIMIT_FORCE_TANGENT_CLOSED_LOOP,
    LIMIT_FORCE_AXIAL_CLOSED_LOOP,
)
import logging

# LOGGER CLASS
This class uses the `logging.Logger` module to create a log using the `%(asctime)s - %(levelname)s - %(message)s` format. Two Handlers are added into it, in order to flow the log both ad the screen (`StreamHandler`) and into a log file (`FileHandler`). The log file is called [isoTdatetime]_testlog.txt

In [ ]:
class MyLogger:
    def __init__(
        self,
        log_file_format: str | None = None,
        formatter: str | None = None,
        log_level: int = 10,  # logging.INFO
        to_logfile: bool = True,
    ):
        self.log_file = (
            log_file_format
            if (log_file_format is not None)
            else f"{datetime.now().isoformat()}_testlog.txt"
        )
        self.formatter = (
            logging.Formatter(formatter)
            if (formatter is not None)
            else logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
        )
        self.logger = logging.Logger(__name__)
        self.logger.setLevel(log_level)

        if to_logfile:
            log_to_file = logging.FileHandler(self.log_file)
            log_to_file.setFormatter(self.formatter)
            self.logger.addHandler(log_to_file)

        log_to_screen = logging.StreamHandler()
        log_to_screen.setFormatter(self.formatter)
        self.logger.addHandler(log_to_screen)

    def info(self, msg: str):
        self.logger.info(msg)

    def warning(self, msg: str):
        self.logger.warning(msg)

    def error(self, msg: str):
        self.logger.error(msg, exc_info=True)

    def critical(self, msg: str):
        self.logger.critical(msg, exc_info=True)

# Retrieve Hardpoint Step Position

In [ ]:
async def get_hardpoint_steps():
    # Hardpoint Ids (!! Starting from 1 !!)
    hardpoint_id = m2.evt_hardpointList.get().actuators

    # Actuator steps lists (both axial and tangent)
    axial_steps = m2.tel_axialActuatorSteps.get().steps
    tangent_steps = m2.tel_tangentActuatorSteps.get().steps

    # Constructing the hardpoint dictionary (e.g. {ids: step_value})
    hardpoint = dict()
    NUM_AXIAL_ACTUATOR = NUM_ACTUATOR - NUM_TANGENT_LINK
    for i, key in enumerate(hardpoint_id):
        if i < 3:
            hardpoint[key] = axial_steps[key - 1]
        else:
            hardpoint[key] = tangent_steps[key - NUM_AXIAL_ACTUATOR - 1]
    return hardpoint

# Command RBP displacement

In [ ]:
async def move_axis(
    logger: logging.Logger, axis: str = None, position: float = 0.0
):

    # ABSOLUTE position displacement
    # Try to move or catch the Exception (i.e. it is in FAULT state)
    # If incorrect axis is provided then raise a ValueError exception

    match axis:
        case "x":
            try:
                await m2.cmd_positionMirror.set_start(x=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case "y":
            try:
                await m2.cmd_positionMirror.set_start(y=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case "z":
            try:
                await m2.cmd_positionMirror.set_start(z=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case "xRot":
            try:
                await m2.cmd_positionMirror.set_start(xRot=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case "yRot":
            try:
                await m2.cmd_positionMirror.set_start(yRot=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case "zRot":
            try:
                await m2.cmd_positionMirror.set_start(zRot=position)
            except Exception as e:
                logger.error("EXCEPTION OCCURRED")
        case _:
            raise ValueError("Unrecognized axis")

In [ ]:
async def wait_m2_to_settle(
    logger: logging.Logger,
    wait_telemetry: float = 0.2,
    settle_time: float = 10.0,
    safety_counter_threshold: int = 5,
):
    # Wait until m2 is settle.
    # Looking at the hardpoints step positions (check the seqNum for the streaming of new data)
    # When at least 'safety_counter_threshold' hardpoints measurements are equal then
    # the M2 is considered settled

    logger.info("Waiting the mirror to settle")
    mirror_is_moving = True

    init_seqnum = m2.tel_axialActuatorSteps.get().private_seqNum
    init_hardpoint = await get_hardpoint_steps()
    logger.info(f"Starting hardpoint condition: \n{init_hardpoint}")
    safety_counter = 0

    while mirror_is_moving:
        await asyncio.sleep(wait_telemetry)
        eval_seqnum = m2.tel_axialActuatorSteps.get().private_seqNum
        if eval_seqnum == init_seqnum:
            # Means no new telemetry is flowing so skip the rest
            # of the loop and wait.
            continue

        init_seqnum = eval_seqnum

        hardpoint = await get_hardpoint_steps()
        logger.info(f"New hardpoint telemetry data: \n{hardpoint}")

        # If the new telemtry is different from the previous one
        # means that M2 is still moving
        if hardpoint != init_hardpoint:
            # setting the current telemetry as the "previous" for the next cycle
            # Skip the rest of the loop
            init_hardpoint = hardpoint
            continue

        # If the previous condition is not met it means that
        # the hardpoints are settled so the mirror as well.
        # As a further safety precaution this has to happen safety_counter_threshold times
        init_hardpoint = hardpoint
        safety_counter += 1

        if safety_counter > safety_counter_threshold:
            logger.info("Position reached")
            mirror_is_moving = False

    # Wait an additional settle time
    await asyncio.sleep(settle_time)

In [ ]:
async def checking_actuator_force(
    logger: logging.Logger,
    wait_force_telemetry: float = 1.0,
    gain: float = 0.95,
):
    # Start checking all the actuator forces.
    # The logic of this block is to gather at least 'min_force_sample' new telemetry measurements,
    # select the maximum reading and check if it is reaching the force limit.
    # In both case (Limit reached or not) the actual force status is logged
    min_force_sample = 10

    # Initialize the force list
    tangent_force_error = list()
    tangent_force = list()
    axial_force = list()
    sum_tangent_force_error = list()
    weight_tangent_force_error = list()

    logger.info("Checking the forces")
    initial_seqnum = m2.tel_forceErrorTangent.get().private_seqNum

    # Loop until the force lists have at least 'min_force_sample' elements
    while len(tangent_force) < min_force_sample:
        eval_seqnum = m2.tel_forceErrorTangent.get().private_seqNum
        if eval_seqnum == initial_seqnum:
            # Means no new telemtry is flowing so skip the rest
            # of the loop and wait.
            await asyncio.sleep(wait_force_telemetry)
            continue

        initial_seqnum = eval_seqnum

        # Here means that new telemtry is flowing, so store the measurement in the force lists
        # Lists of lists
        tangent_force_error.append(m2.tel_forceErrorTangent.get().force)
        tangent_force.append(m2.tel_tangentForce.get().measured)
        axial_force.append(m2.tel_axialForce.get().measured)

        # Lists of floats
        sum_tangent_force_error.append(abs(m2.tel_forceErrorTangent.get().sum))
        weight_tangent_force_error.append(
            abs(m2.tel_forceErrorTangent.get().weight)
        )

        await asyncio.sleep(wait_force_telemetry)

    logger.info("Force telemetry data acquisition done")

    # Get the maximum value of each list and save also its index.
    # By means it is possible to log the measurement of all forces, containing the maximum.
    max_tangent_force_error = [
        np.unravel_index(
            np.abs(np.array(tangent_force_error)).argmax(),
            np.array(tangent_force_error).shape,
        ),
        np.abs(np.array(tangent_force_error)).max(),
    ]
    max_tangent_force = [
        np.unravel_index(
            np.abs(np.array(tangent_force)).argmax(),
            np.array(tangent_force).shape,
        ),
        np.abs(np.array(tangent_force)).max(),
    ]
    max_axial_force = [
        np.unravel_index(
            np.abs(np.array(axial_force)).argmax(), np.array(axial_force).shape
        ),
        np.abs(np.array(axial_force)).max(),
    ]
    max_sum_tangent_force_error = [
        np.unravel_index(
            np.abs(np.array(sum_tangent_force_error)).argmax(),
            np.array(sum_tangent_force_error).shape,
        ),
        np.abs(np.array(sum_tangent_force_error)).max(),
    ]
    max_weight_tangent_force_error = [
        np.unravel_index(
            np.abs(np.array(weight_tangent_force_error)).argmax(),
            np.array(weight_tangent_force_error).shape,
        ),
        np.abs(np.array(weight_tangent_force_error)).max(),
    ]

    # Controlling if the maximum exceed the Threshold.
    # N.B. The threshold setted here are below the real force threshold
    limit_reach = True
    max_id = -1

    if (
        max_tangent_force_error[1] > gain * TANGENT_LINK_LOAD_BEARING_LINK
    ):  # 950:
        max_id = max_tangent_force_error[0][0]

    elif (
        max_tangent_force[1] > gain * LIMIT_FORCE_TANGENT_CLOSED_LOOP
    ):  # 4870:
        max_id = max_tangent_force[0][0]

    elif (
        max_sum_tangent_force_error[1] > gain * TANGENT_LINK_THETA_Z_MOMENT
    ):  # 950:
        max_id = sum_tangent_force_error[0][0]

    elif (
        max_weight_tangent_force_error[1]
        > gain * TANGENT_LINK_TOTAL_WEIGHT_ERROR
    ):  # 1900:
        max_id = weight_tangent_force_error[0][0]

    elif max_axial_force[1] > gain * LIMIT_FORCE_AXIAL_CLOSED_LOOP:  # 420:
        max_id = max_axial_force[0][0]

    else:
        limit_reach = False

    return (
        limit_reach,
        max_id,
        tangent_force_error,
        tangent_force,
        axial_force,
        sum_tangent_force_error,
        weight_tangent_force_error,
    )

In [ ]:
# Function that managed the M2 rigid body movement, it calls first the move_axis()
# function to perform the displacement, then the wait_m2_to_settle() wait and check
# until the completion of such movement. Eventually, the checking_actuator_force()
# controls all the forces and stop if some threshold is met.


async def move_m2_rbp(
    logger: logging.Logger, axis: str = None, position: float = 0.0
):
    if "Rot" in axis:
        logger.info(f"Moving to {position} arcsec along {axis}-axis")
    else:
        logger.info(f"Moving to {position} micron along {axis}-axis")
    await move_axis(logger, axis, position)
    await wait_m2_to_settle(logger)

    (
        limit_reach,
        max_id,
        tangent_force_error,
        tangent_force,
        axial_force,
        sum_tangent_force_error,
        weight_tangent_force_error,
    ) = await checking_actuator_force(logger)

    log = (
        f"\nMeasured tangent force(<4893): {[round(el, 2) for el in tangent_force[max_id]]}"
        f"\nForce tangent error (<1000): {[round(el, 2) for el in tangent_force_error[max_id]]}"
        f"\nSum tangent force error(<1000): {sum_tangent_force_error[max_id]:.2f}"
        f"\nWeight tangent force error(<2000): {weight_tangent_force_error[max_id]:.2f}"
        f"\nMax measured axial force(<489): {[round(el, 2) for el in axial_force[max_id]]}"
        f"\nM2 LAST POSITION:"
        f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
        f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
        f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
        f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
        f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
        f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
    )

    if limit_reach:
        logger.warning("LIMIT REACHED")
        logger.warning(log)

    else:
        logger.info("CURRENT STATUS")
        logger.info(log)

    return limit_reach

In [ ]:
# Timeout coroutine
async def timeout(timeout: float):
    await asyncio.sleep(timeout)
    return

In [ ]:
# Function for query to user the intention to start (yes or no)
async def start_m2_move():
    start = await asyncio.to_thread(input, "Start (y or n)")
    match start:
        case "y":
            return True
        case _:
            return False

## N.B: CONSERVATIVE LIMIT FOR M2 TANGENT FORCE ERROR.
Based on previous multiple faults of m2 during rigid body movements a maximum displacement has been implement (the `max_step` parameter).

For more information see the [SITCOMTN-99](https://github.com/lsst-sitcom/sitcomtn-099)

In [ ]:
# Main Coroutine that call into two different threads the timeout coroutine and the Start query function.
# This function return as soon as one of the two coroutine ends.
# This implement a timeout for the Start query function.
async def main(
    axis: str,
    logger: logging.Logger,
    max_position: float = None,
    increment: float = None,
):
    t = 120  # timeout time in sec

    if "Rot" in axis:
        max_step = 50  # Conservative value to avoid Tangent Force Error fault.
    else:
        max_step = 200  # Conservative value to avoid Tangent Force Error fault.

    task1 = asyncio.create_task(timeout(t), name="Timeout")
    task2 = asyncio.create_task(start_m2_move(), name="M2 movement")

    done, pending = await asyncio.wait(
        [task1, task2], return_when=asyncio.FIRST_COMPLETED
    )

    task_done = None
    res = None

    for el in done:
        task_done = el.get_name()
        res = el.result()

    for el in pending:
        el.cancel()

    if task_done == "M2 movement" and res:
        position = input("Insert position")

        match axis:
            case "x":
                current_position = m2.tel_position.get().x
            case "y":
                current_position = m2.tel_position.get().y
            case "z":
                current_position = m2.tel_position.get().z
            case "xRot":
                current_position = m2.tel_position.get().xRot
            case "yRot":
                current_position = m2.tel_position.get().yRot
            case "zRot":
                current_position = m2.tel_position.get().zRot
            case _:
                raise ValueError("Unrecognized axis")

        if len(position) == 0:
            logger.warning("Position invalid, quit test.")
            return True
        elif abs(current_position - float(position)) > max_step:
            logger.warning(
                "The request position is beyond the safe movement range"
            )
            return True

        limit_reach = await move_m2_rbp(logger, axis, float(position))
        return limit_reach
    else:
        return True

## Prepare the M2 CSC and put to Enabled state

In [ ]:
logger = MyLogger()

domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")

In [ ]:
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

# SYSTEM STATUS LEGEND    
1. DISABLED = 1
2. ENABLED = 2
3. FAULT = 3
4. OFFLINE = 4
5. STANDBY = 5

**Fault restoring cycle**

Fault --> Standby  -->  Disable  -->  Enabled

**To Stop**

Enabled  -->  Disable  -->  Standby

In [ ]:
# Get system status
state = m2.evt_summaryState.get()
logger.info(f"System Status: {state.summaryState}")

# Fault Restoring Cycle

In [ ]:
# Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=550)

# Stop Cycle

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
# Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

## LUT FILES
1. Configurable_File_Description_20180831T091922_M2_optical.csv
2. Configurable_File_Description_20180831T092326_M2_handling.csv
3. Configurable_File_Description_20180831T092423_surrogate_optical.csv
4. Configurable_File_Description_20180831T092556_surrogate_handling.csv

In [ ]:
ccfile = m2.evt_config.get().get_vars()["configuration"]
print(ccfile)

In [ ]:
# Setting the right LUT ***ONLY IN DISABLE STATE ***
ccfile = m2.evt_config.get().get_vars()["configuration"]

if "M2_optical" not in ccfile:
    config_files = (
        m2.evt_configurationFiles.get().get_vars()["files"].split(",")
    )
    ncfile = [el for el in config_files if "M2_optical" in el][0]
    await m2.cmd_setConfigurationFile.set_start(file=ncfile)
    logger.info(
        f'Current config file (switch to enable state to make the change effective): {m2.evt_config.get().get_vars()["configuration"]}'
    )

In [ ]:
# Debug cell
print(m2.evt_interlock.get())
print(m2.evt_innerLoopControlMode.get())
print(m2.evt_errorCode.get())
print(m2.evt_forceBalanceSystemStatus.get())

In [ ]:
# X-DOF

axis = "x"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}"
    f"\nY: {m2.tel_position.get().y:.2f}"
    f"\nZ: {m2.tel_position.get().z:.2f}"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)

In [ ]:
# Y-DOF

axis = "y"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
    f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
    f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)

In [ ]:
# Z-DOF

axis = "z"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
    f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
    f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)

In [ ]:
# xRot-DOF

axis = "xRot"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
    f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
    f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)

In [ ]:
# yRot-DOF

axis = "yRot"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
    f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
    f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)

In [ ]:
# zRot-DOF

axis = "zRot"

log = (
    f"** STARTING MOVING ON {axis}-axis**"
    f"\nM2 POSITION:"
    f"\nX: {m2.tel_position.get().x:.2f}, $\mu$m"
    f"\nY: {m2.tel_position.get().y:.2f}, $\mu$m"
    f"\nZ: {m2.tel_position.get().z:.2f}, $\mu$m"
    f"\nXROT: {m2.tel_position.get().xRot:.2f}, arcsec"
    f"\nYROT: {m2.tel_position.get().yRot:.2f}, arcsec"
    f"\nZROT: {m2.tel_position.get().zRot:.2f}, arcsec"
)

logger.info(log)

keep_continue = False
while not keep_continue:
    keep_continue = await main(axis=axis, logger=logger)